In [1]:
import pandas as pd
import urbanaccess as ua
import pandana as pdna
import osmnet as osm
import urbanaccess.plot
import pandana.network

In [2]:
gtfs_df = ua.gtfs.load.gtfsfeed_to_df(r'../../gtfs_data/gtfs_bus')

GTFS text file encoding check completed. Took 0.26 seconds
GTFS text file header whitespace check completed. Took 0.88 seconds
The unique agency id: metro_-_los_angeles was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.16 seconds
Appended route type to stops
Appended route type to stop_times
--------------------------------
Successfully converted ['departure_time', 'arrival_time'] to seconds past midnight and appended new columns to stop_times. Took 14.22 seconds
1 GTFS feed files successfully read as dataframes: ['../../gtfs_data/gtfs_bus']. Took 20.11 seconds


In [3]:
ua_net = ua.gtfs.network.create_transit_net(gtfs_df, 
                                   day='monday', 
                                   timerange=["00:00:00","03:00:00"], 
                                   overwrite_existing_stop_times_int=False, 
                                   use_existing_stop_times_int=True, 
                                   save_processed_gtfs=False, 
                                   save_dir='data', 
                                   save_filename=None)

Using calendar to extract service_ids to select trips.
29 service_ids were extracted from calendar
14,291 trip(s) 43.922 percent of 32,537 total trip records were found in calendar
NOTE: 18,246 trip(s) 56.078 percent of 32,537 total trip records were found to be in calendar_dates and not calendar. The calendar_dates_lookup parameter is None. In order to use the trips inside the calendar_dates dataframe it is suggested you specify a search parameter in the calendar_dates_lookup dict. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify the service_ids. When in doubt do not use the calendar_dates_lookup parameter.
14,291 of 32,537 total trips were extracted representing calendar day: monday. Took 0.08 seconds
There are no departure time records missing from trips following monday schedule. There are no records to interpolate.
Difference between stop times has been successfully calculated. Took 2.31 seconds
Stop times from

In [4]:
ua.gtfs.headways.headways(gtfs_df,["00:00:00","03:00:00"])

Stop times from 00:00:00 to 03:00:00 successfully selected 68 records out of 857,890 total records (0.01 percent of total). Took 0.01 seconds
Starting route stop headway calculation for 68 route stops...
Route stop headway calculation complete. Took 0.15 seconds
headway calculation complete. Took 1.90 seconds


In [5]:
osm_data = ua.osm.load.ua_network_from_bbox(bbox=(-118.85, 33.67, -117.67, 34.32),
                                           network_type='walk', 
                                           timeout=180, 
                                           memory=None, 
                                           max_query_area_size=2500000000L, 
                                           remove_lcn=True)
#$$c(W 118°39'00"--W 117°41'00"/N 34°19'00"--N 33°40'00")

Requesting network data within bounding box from Overpass API in 6 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](33.67000000,-118.06402707,34.00108011,-117.66998927);>;);out;'}"
Downloaded 69,570.9KB from www.overpass-api.de in 20.57 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](33.67124676,-118.46050146,33.99830286,-118.05998182);>;);out;'}"
Downloaded 48,671.7KB from www.overpass-api.de in 19.58 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["ped

In [6]:
all_net = ua.osm.network.create_osm_net(osm_edges=osm_data[1], osm_nodes=osm_data[0], travel_speed_mph=3, network_type='walk')

Created OSM network with travel time impedance using a travel speed of 3 MPH. Took 0.11 seconds


In [7]:
all_net.transit_nodes = ua_net.transit_nodes
all_net.transit_edges = ua_net.transit_edges
ua_integrated_net = ua.network.integrate_network(all_net, 
                             headways=True, 
                             urbanaccess_gtfsfeeds_df=gtfs_df, 
                             headway_statistic='mean')

Loaded UrbanAccess network components comprised of: 68 transit nodes and 66 edges; 406,736 OSM nodes and 1,146,542 edges.
routes successfully joined to transit nodes. Took 0.42 seconds
Connector edges between the OSM and transit network nodes successfully completed. Took 0.56 seconds
mean route stop headway will be used for pedestrian to transit edge impedance.
headway impedance calculation completed. Took 0.01 seconds
Fixed unicode error in name column
Edge and node tables formatted for Pandana
Network edge and node network integration completed successfully resulting in a total of 406,804 nodes and 1,146,744 edges: Transit: 68 nodes 66 edges; OSM 406,736 nodes 1,146,542 edges; and 136 connector edges.


In [8]:
ua.network.save_network(urbanaccess_network=ua_net,
                        filename='net_late_night/net_late_night_ua.h5',
                        overwrite_key = True);

New net_late_night/net_late_night_ua.h5 hdf5 store created in dir: data
edges saved in data/net_late_night/net_late_night_ua.h5 hdf5 store.
Using existing data/net_late_night/net_late_night_ua.h5 hdf5 store.
nodes saved in data/net_late_night/net_late_night_ua.h5 hdf5 store.


In [11]:
fig,ax = urbanaccess.plot.plot_net(nodes=ua_integrated_net.net_nodes,
                          edges=ua_integrated_net.net_edges, 
                          x_col='x', 
                          y_col='y', 
                          bbox=(-118.20, 33.90, -118.00, 34.00), 
                          fig_height=6, 
                          margin=0.02, 
                          edge_color='#999999', 
                          edge_linewidth=1, 
                          edge_alpha=1, 
                          node_color='black', 
                          node_size=15, 
                          node_alpha=1,
                          node_edgecolor='none', 
                          node_zorder=3,
                          nodes_only=False)

Figure created. Took 130.98 seconds


In [12]:
fig.savefig("./data/net_late_night/small_fig.png")

In [9]:
imp = pd.DataFrame(ua_integrated_net.net_edges['weight'])
net = pdna.network.Network(ua_integrated_net.net_nodes.x, ua_integrated_net.net_nodes.y, 
                                ua_integrated_net.net_edges.from_int, ua_integrated_net.net_edges.to_int, imp, False)

In [10]:
net.save_hdf5("./data/net_late_night/net_late_night.h5")

In [30]:
ua_integrated_net.osm_nodes

,x,y,id,net_type
id,,,,
653656,-118.377054,33.959841,653656,walk
653661,-118.369066,33.961598,653661,walk
653688,-118.429509,34.027042,653688,walk
653689,-118.425472,34.028633,653689,walk
653690,-118.421842,34.030118,653690,walk
653691,-118.419977,34.031146,653691,walk
653692,-118.418453,34.031974,653692,walk
653693,-118.417575,34.032450,653693,walk
668243,-118.352539,33.945462,668243,walk


In [28]:
net.nodes_df

,x,y
id_int,,
1,-118.541748,34.172424
2,-118.542587,34.171299
3,-118.538841,34.169781
4,-118.535866,34.169090
5,-118.535851,34.172226
6,-118.530487,34.172241
7,-118.526970,34.172161
8,-118.521706,34.172142
9,-118.518196,34.172180
